# Introduction and Objective
The notebook begins with a brief introduction describing its purpose: to extract data from a CSV file containing information about traffic accidents. The main goal is to load this data into a PostgreSQL database for further analysis.

## Environment Setup
The first step is to set up the environment to ensure that the necessary libraries and modules are available. sys.path.append is used to add the parent directory path to the system, allowing the import of custom modules, such as conexion_db.py, which is used to establish the connection with the database. 

In [11]:
import sys
import os
import importlib    
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
importlib.invalidate_caches()
from config.conexion_db import conectar_db

ModuleNotFoundError: No module named 'config.conexion_db'

## Importing libraries and modules

The necessary libraries for data processing and database interaction are imported:

- sqlalchemy: To execute SQL queries.
- pandas: For data manipulation and analysis.

In [30]:
from sqlalchemy import text
import pandas as pd

import psycopg2

def conectar_db():
    try:
        conn = psycopg2.connect(
            dbname="public",
            user="postgres",
            password="postgres",
            host="localhost",
            port="5432",
            options="-c client_encoding=UTF8"
        )
        conn.set_client_encoding('UTF8')  # Asegurar que la codificación sea UTF-8
        print("✅ Conexión exitosa con psycopg2")
        return conn
    except Exception as e:
        print("❌ Error en conexión con psycopg2:", e)
        return None

## Reading the Dataset
The CSV file is loaded into a pandas DataFrame using pd.read_csv. The file contains 209,306 rows and 24 columns, with detailed information about each accident, such as the date, weather conditions, type of collision, reported injuries, and more.

In [31]:
# Cargar el CSV asegurando que el delimitador sea correcto
df = pd.read_csv("../data/raw/crashtraffic.csv", encoding='utf-8')
df

,crash_date,traffic_control_device,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,road_defect,crash_type,...,injuries_fatal,injuries_incapacitating,injuries_non_incapacitating,injuries_reported_not_evident,injuries_no_indication,crash_hour,crash_day_of_week,crash_month,Start_Lat,Start_Lng
0,07/29/2023 01:00:00 PM,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,UNKNOWN,UNKNOWN,NO INJURY / DRIVE AWAY,...,0.0,0.0,0.0,0.0,3.0,13.0,7.0,7.0,34.751225,-86.755043
1,08/13/2023 12:11:00 AM,TRAFFIC SIGNAL,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,...,0.0,0.0,0.0,0.0,2.0,0.0,1.0,8.0,36.124142,-86.750488
2,12/09/2021 10:30:00 AM,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,REAR END,T-INTERSECTION,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,...,0.0,0.0,0.0,0.0,3.0,10.0,5.0,12.0,36.134838,-86.820877
3,08/09/2023 07:55:00 PM,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,ANGLE,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,...,0.0,0.0,5.0,0.0,0.0,19.0,4.0,8.0,33.751957,-84.495995
4,08/19/2023 02:55:00 PM,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,REAR END,T-INTERSECTION,STRAIGHT AND LEVEL,UNKNOWN,UNKNOWN,NO INJURY / DRIVE AWAY,...,0.0,0.0,0.0,0.0,3.0,14.0,7.0,8.0,35.162975,-80.889984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209301,09/13/2023 01:08:00 PM,UNKNOWN,UNKNOWN,UNKNOWN,TURNING,FOUR WAY,STRAIGHT AND LEVEL,UNKNOWN,UNKNOWN,NO INJURY / DRIVE AWAY,...,0.0,0.0,0.0,0.0,2.0,13.0,4.0,9.0,34.805672,-82.230499
209302,07/18/2023 02:10:00 PM,UNKNOWN,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,NOT DIVIDED,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NO INJURY / DRIVE AWAY,...,0.0,0.0,0.0,0.0,2.0,14.0,3.0,7.0,39.174740,-119.740425
209303,10/23/2019 01:32:00 PM,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,PEDESTRIAN,DIVIDED - W/MEDIAN (NOT RAISED),STRAIGHT ON GRADE,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,...,0.0,2.0,0.0,0.0,0.0,13.0,4.0,10.0,39.492794,-119.784584
209304,06/01/2020 03:23:00 PM,NO CONTROLS,CLEAR,DAYLIGHT,PEDESTRIAN,T-INTERSECTION,STRAIGHT AND LEVEL,DRY,NO DEFECTS,INJURY AND / OR TOW DUE TO CRASH,...,0.0,0.0,1.0,0.0,1.0,15.0,2.0,6.0,36.110260,-86.721001


## Connecting to PostgreSQL and Creating the Table
A connection to the PostgreSQL database is established using the conectar_db function from the conexion_db module. Then, a table named accidentes is created with a structure that mirrors the columns of the DataFrame. The table includes fields such as crash_date, weather_condition, traffic_control_device, and various fields related to injuries and the type of accident.

In [32]:
conexion = conectar_db()

❌ Error en conexión con psycopg2: 


In [26]:


def create_table():
    conexion = conectar_db()
    if conexion is None:
        print("❌ No se pudo establecer la conexión. Abortando creación de tabla.")
        return
    cursor = conexion.cursor()


    cursor.execute("""
        CREATE TABLE IF NOT EXISTS accidentes (
            id SERIAL PRIMARY KEY,
            crash_date TIMESTAMP,
            traffic_control_device TEXT,
            weather_condition TEXT,
            lighting_condition TEXT,
            first_crash_type TEXT,
            trafficway_type TEXT,
            alignment TEXT,
            roadway_surface_cond TEXT,
            road_defect TEXT,
            crash_type TEXT,
            intersection_related CHAR(1),
            damage TEXT,
            prim_contributory_cause TEXT,
            num_units INT,
            most_severe_injury VARCHAR,
            injuries_total FLOAT,
            injuries_fatal FLOAT,
            injuries_incapacitating FLOAT,
            injuries_non_incapacitating FLOAT,
            injuries_reported_not_evident FLOAT,
            injuries_no_indication FLOAT,
            crash_hour INT,
            crash_day_of_week INT,
            crash_month INT,
            Start_Lat INT,
            Start_Lng INT
        );
    """)

    conexion.commit()
    print("Tabla 'accidentes' creada exitosamente.")

    cursor.close()
    conexion.close()


if __name__ == "__main__":
    create_table()

✅ Conexión exitosa con psycopg2
Tabla 'accidentes' creada exitosamente.


## Data Transformation
Before loading the data into the database, some transformations are performed:

Date conversion: The crash_date column is converted to a date-time format (datetime) using pd.to_datetime.

Handling null values: NaN values are converted to None to avoid issues when inserting data into PostgreSQL.

In [27]:
df['crash_date'] = pd.to_datetime(df['crash_date'], format='%m/%d/%Y %I:%M:%S %p')

# Convertir NaN a None para evitar problemas en PostgreSQL
df = df.where(pd.notna(df), None)

 ## Loading Data into PostgreSQL
The data is inserted into the accidentes table using the executemany function from psycopg2. This allows multiple rows to be inserted in a single operation, which is efficient for large datasets.

In [28]:


conexion = conectar_db()
cursor = conexion.cursor()

query = """
    INSERT INTO accidentes (
        crash_date, traffic_control_device, weather_condition, lighting_condition,
        first_crash_type, trafficway_type, alignment, roadway_surface_cond,
        road_defect, crash_type, intersection_related, damage,
        prim_contributory_cause, num_units, most_severe_injury,
        injuries_total, injuries_fatal, injuries_incapacitating,
        injuries_non_incapacitating, injuries_reported_not_evident,
        injuries_no_indication, crash_hour, crash_day_of_week, crash_month, Start_Lat, Start_Lng
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s);
"""

# Convertir el DataFrame en una lista de tuplas
data = df.values.tolist()

# Insertar los datos en la tabla
cursor.executemany(query, data)  

# Confirmar la transacción
conexion.commit()
print("Datos insertados exitosamente.")

# Cerrar la conexión
cursor.close()
conexion.close()


✅ Conexión exitosa con psycopg2
Datos insertados exitosamente.


## Verifying Data Transfer
Finally, the data transfer is verified by executing an SQL query to select the first 5 rows from the accidentes table and displaying the result in a DataFrame. This confirms that the data has been transferred correctly and that the table structure is as expected.

In [29]:

conexion = conectar_db()

df_verificacion = pd.read_sql("SELECT * FROM accidentes LIMIT 5;", conexion)
df_verificacion


✅ Conexión exitosa con psycopg2


C:\Users\57302\AppData\Local\Temp\ipykernel_30092\1519445756.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_verificacion = pd.read_sql("SELECT * FROM accidentes LIMIT 5;", conexion)


,id,crash_date,traffic_control_device,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,roadway_surface_cond,road_defect,...,injuries_fatal,injuries_incapacitating,injuries_non_incapacitating,injuries_reported_not_evident,injuries_no_indication,crash_hour,crash_day_of_week,crash_month,start_lat,start_lng
0,1,2023-07-29 13:00:00,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,UNKNOWN,UNKNOWN,...,0.0,0.0,0.0,0.0,3.0,13,7,7,35,-87
1,2,2023-08-13 00:11:00,TRAFFIC SIGNAL,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,...,0.0,0.0,0.0,0.0,2.0,0,1,8,36,-87
2,3,2021-12-09 10:30:00,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,REAR END,T-INTERSECTION,STRAIGHT AND LEVEL,DRY,NO DEFECTS,...,0.0,0.0,0.0,0.0,3.0,10,5,12,36,-87
3,4,2023-08-09 19:55:00,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,ANGLE,FOUR WAY,STRAIGHT AND LEVEL,DRY,NO DEFECTS,...,0.0,0.0,5.0,0.0,0.0,19,4,8,34,-84
4,5,2023-08-19 14:55:00,TRAFFIC SIGNAL,CLEAR,DAYLIGHT,REAR END,T-INTERSECTION,STRAIGHT AND LEVEL,UNKNOWN,UNKNOWN,...,0.0,0.0,0.0,0.0,3.0,14,7,8,35,-81
